# **Experiment 1**: proof of concent from ground truth
Vary noise level and number of images

In [1]:
# Mount Google Drive
from google.colab import drive # import drive from google colab
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/PhD/Projects/2 - Cryo-EM/src/Cryo-EM

/content/drive/MyDrive/PhD/Projects/2 - Cryo-EM/src/Cryo-EM


In [3]:
# install libraries in Colab env
!pip install mrcfile
!pip install aspire
!pip install quaternionic
!pip install pykeops[colab] > install.log

In [4]:
import numpy as np
import os
import logging

from projects.rkhs_lifting.experiments.experimentA1.E1_run import run_experiment
from projects.rkhs_lifting.experiments.experimentA1.E2_post_processing import post_processing

from tools.exp_tools import Exp

logger = logging.getLogger(__name__)

In [6]:
# Experiment parameters
img_size = 65  # was 65 before and was 129 in rotation estimation paper
mr_repeat = 1 # number of times mesh refinement
snr = 1 / 2  # [1/2, 1/4, 1/8, 1/16, 1/32, 1/64, 1/128, 1/256]
num_imgs = 512  # [100, 500, 1000]
k = 40
kernel_radius = np.pi/k  # radius of kernel

# Directly start experiment
exp = Exp()

exp.begin(prefix="expA1" , postfix="L{}_N{}_r{}_k{}".format(img_size,num_imgs,mr_repeat,k))
exp.dbglevel(4)

2021-10-08 11:18:27,716 INFO Results folder is results/expA1_21-10-08_11-18-27_L65_N512_r1_k40


In [7]:
# Set data path
data_dir = "data"
data_filename = "clean70SRibosome_vol_65p.mrc"
data_path = os.path.join(data_dir, data_filename)

# Set results folder if skip_preprocessing
# results_folder = "results/lifting_21-04-21_11-09-45"  # "results/lifting_21-04-26_12-22-06"

In [8]:
# Experiment

logger.info("Start Experiment")
logger.info("Running Experiment for SNR = {} and {} images".format(snr, num_imgs))

run_experiment(exp=exp,
               num_imgs=num_imgs,
               snr=snr,
               img_size=img_size,
               kernel_radius=kernel_radius,
               mr_repeat=mr_repeat,
               data_path=data_path
               )

2021-10-08 11:18:34,960 INFO Start Experiment
2021-10-08 11:18:34,963 INFO Running Experiment for SNR = 0.5 and 512 images
2021-10-08 11:18:34,967 INFO This experiment illustrates orientation refinement using a lifting approach
2021-10-08 11:18:34,969 INFO Initialize simulation object and CTF filters.
2021-10-08 11:18:34,971 INFO Load 3D map and downsample 3D map to desired grids of 65 x 65 x 65.
2021-10-08 11:18:34,990 INFO Use downsampled map to creat simulation object.
2021-10-08 11:18:35,021 INFO Get true rotation angles generated randomly by the simulation object.
2021-10-08 11:18:35,219 INFO Trying NFFT backend finufft
2021-10-08 11:18:35,228 INFO NFFT backend finufft usable.
2021-10-08 11:18:35,231 INFO Trying NFFT backend cufinufft
2021-10-08 11:18:35,238 INFO NFFT backend cufinufft not usable:
	No module named 'pycuda'
2021-10-08 11:18:35,239 INFO Trying NFFT backend pynfft
2021-10-08 11:18:35,248 INFO NFFT backend pynfft not usable:
	No module named 'pynfft'
2021-10-08 11:18:

RuntimeError: ignored

In [ ]:
logger.info("Start Postprocessing")
# Postprocessing
post_processing(exp=exp,
               num_imgs=num_imgs,
               snr=snr,
              #  results_folder=results_folder #?
               )
